In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
%matplotlib notebook
from tqdm import tqdm #プログレスバー
import csv
from MOEAD import *
import random

In [2]:
class Knapsack:
    def __init__(self, items_file_name, knapsack, item_num):
        #物品データ読み込み
        with open(items_file_name, 'r') as f:
            line = f.readline()
            print(line)
            self.names = []
            self.weights = []
            self.profits = []
            self.capacities = []
            for k in range(knapsack):
                line = f.readline() # "="
                if not("=" in line):
                    print("読み込みが正しく行えませんでした k:%d, i:%d" % (k, i))
                    exit()
                line = f.readline() # "knapsack n:"
                print(line)
                line = f.readline() # "capacity"
                self.capacities.append(float(line[11:]))
                name = []
                weight = []
                profit = []
                for i in range(item_num):
                    line = f.readline() # "item"
                    name.append(line[1:-1])
                    line = f.readline() # "weight"
                    weight.append(int(line[10:]))
                    line = f.readline() # "profit"
                    profit.append(int(line[10:]))
                self.names.append(np.array(name))
                self.weights.append(np.array(weight))
                self.profits.append(np.array(profit))
                print(name[0:10])
                print(weight[0:10])
                print(profit[0:10])
                
        self.sum_profit = []
        for i in range(knapsack):
            self.sum_profit.append(sum(self.profits[i]))

In [3]:
# メイン処理記述
items_file_name = "knapsack_500_2.txt"

H = 200 # 分割数パラメータ
T = 10 #近傍数
G = 10 # 世代数
m = 2 #目的数
n = 500 #アイテム数
phi = 0.5 # Φ 実行可能率
p_m = 1.0 / n # 突然変異率

In [4]:
knapsack = Knapsack(items_file_name, m, n)

knapsack problem specification (2 knapsacks, 500 items)

knapsack 1:

['item 1:', 'item 2:', 'item 3:', 'item 4:', 'item 5:', 'item 6:', 'item 7:', 'item 8:', 'item 9:', 'item 10:']
[68, 80, 93, 14, 89, 15, 37, 41, 76, 26]
[100, 53, 48, 38, 72, 70, 75, 11, 87, 70]
knapsack 2:

['item 1:', 'item 2:', 'item 3:', 'item 4:', 'item 5:', 'item 6:', 'item 7:', 'item 8:', 'item 9:', 'item 10:']
[55, 18, 80, 62, 75, 97, 54, 70, 11, 44]
[59, 31, 75, 83, 92, 17, 22, 65, 45, 67]


In [5]:
#突然変異
def mutation_func(x, mutation):
    for i in range(len(x)):
        if random.random() < mutation:
            x[i] = (x[i] + 1) % 2
    return x

# 評価関数
def func_1(x):
    weight = knapsack.weights[0] * x
    weight = sum(weight.tolist())
    profit = knapsack.profits[0] * x
    profit = sum(profit.tolist())
    
    #print("weight:%d, profit:%d" % (weight,profit))
        
    # 修復法
    while knapsack.capacities[0] < weight:
        is_not_changed = True
        while is_not_changed:
            i = random.randrange(n)
            if x[i] == 1:
                x[i] = 0
                weight = weight - knapsack.weights[0][i]
                profit = profit - knapsack.profits[0][i]
                is_not_changed = False
                
    return knapsack.sum_profit[0] - profit
                
def func_2(x):
    weight = knapsack.weights[1] * x
    weight = sum(weight.tolist())
    profit = knapsack.profits[1] * x
    profit = sum(profit.tolist())
        
    # 修復法
    while knapsack.capacities[1] < weight:
        is_not_changed = True
        while is_not_changed:
            i = random.randrange(n)
            if x[i] == 1:
                x[i] = 0
                weight = weight - knapsack.weights[1][i]
                profit = profit - knapsack.profits[1][i]
                is_not_changed = False
                
    return knapsack.sum_profit[1] - profit
        

fs = [func_1, func_2]

# x初期化関数
def init_x_func(L):
    x = [knapsack.]
    for i in range(len(L)):
        x_i = []
        for j in range(n):
            # 0.7の確率で0
            x_i.append(0 if random.random() < 0.7 else 1)
        x.append(x_i)
    return x

In [6]:
cross_over = OnePointCrossOver(n, int(n/2))
mutation = Mutation(mutation_func ,p_m)
moead = MOEAD(m, H, T, fs, cross_over, mutation)
moead.set_init_x_func(init_x_func)

m:2, H:200, N:201, T:10


In [7]:
moead.init_phase()
x = moead.solution_search_phase(G)

100%|██████████| 10/10 [00:01<00:00,  5.59it/s]


In [9]:
solution_x_file_name = "solution_x.csv"
solution_profit_weight_file_name = "solution_profit_weight.csv"

weights = []
profits = []
for x_j in x:
    weights.append([sum((knapsack.weights[0] * x_j).tolist()), sum((knapsack.weights[1] * x_j).tolist())])
    profits.append([sum((knapsack.profits[0] * x_j).tolist()), sum((knapsack.profits[1] * x_j).tolist())])

with open(solution_x_file_name, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(knapsack.names[0])
    writer.writerow(knapsack.weights[0])
    writer.writerow(knapsack.profits[0])
    writer.writerow(knapsack.names[1])
    writer.writerow(knapsack.weights[1])
    writer.writerow(knapsack.profits[1])
    for x_j in x:
        writer.writerow(x_j)
        
with open(solution_profit_weight_file_name, 'w') as f:
    writer = csv.writer(f)
    writer.writerow([("profit%d" % i) for i in range(m)] + [("weight%d" % i) for i in range(m)])
    writer.writerow(knapsack.sum_profit + knapsack.capacities)
    writer.writerow([])
    for i in range(len(profits)):
        writer.writerow(profits[i] + weights[i])